I want to do a quick pearson's chi squared independence test between the independent variables and dependent variable, just after still up to this point not producing much benefit from the last models. This should at least in a super rudimentary way help understand whether the variables are at all useful.



In [1]:
import pandas as pd
import xgboost as xgb
from xgboost import XGBClassifier
import datetime; import pytz
import matplotlib.pyplot as plt
# from scipy.special import softmax
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split # (*arrays, **options)
import numpy as np
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, LabelBinarizer
from sklearn.datasets import load_svmlight_file
from joblib import dump, load
import joblib
import os
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score
import fresh.utils as fu

from importlib import reload
from collections import Counter
from tqdm.notebook import tqdm
import fresh.preproc.v1 as pv1
import fresh.preproc.v2 as pv2

In [2]:
localdir = '/home/ec2-user/SageMaker/learn-citibike'  # sagemaker
datadir = f'{localdir}/artifacts/2020-07-03T171842Z'

train_loc = f'{datadir}/train.libsvm'

# Convert the dtrain to numpy  ( nice advice from https://stackoverflow.com/a/40430328  )
train_data = load_svmlight_file(train_loc)
X_train = train_data[0].toarray()
y_train = train_data[1]

In [3]:
rawdatadir = f'{localdir}/local_datas'

tripsdf = pd.read_csv(f'{rawdatadir}/2013-07 - Citi Bike trip data.csv'
                     )
stationsdf = pd.read_csv(f'{localdir}/datas/stations/stations-2018-12-04-c.csv',
                        index_col=0)
X, y, neighborhoods = fu.prepare_data(tripsdf, stationsdf)


In [42]:
# For my own sanity checking... 
# gender vs handedness , example from https://en.wikipedia.org/wiki/Contingency_table
# Also some inspiration on which statistic is good in my situation, 
#      from here , https://stackabuse.com/statistical-hypothesis-analysis-in-python-with-anovas-chi-square-and-pearson-correlation/
# one more reference that helped me : https://www.ling.upenn.edu/~clight/chisquared.htm
def chi_sq_test(table):
    chi2, p, dof, expected = chi2_contingency(table)
    print(f'chi2= {chi2}, p= {p}, dof= {dof}')
    # print(expected)
    print('Ho, our null hypothesis is the two variables are independent.')
    if p <= 0.05:
        print('p value <= 0.05 so can lean towards rejecting the null hypothesis.\n'
             '=> some dependency likely')
    else:
        print('p value > 0.05 so we likely cannot reject the null hypothesis \n'
             '=> independent')
    print('\n')
    
table = np.array([[43, 9], 
                  [44, 4]])
chi_sq_test(table)


table = np.array([[43, 40], 
                  [44, 4]])
chi_sq_test(table)


chi2= 1.0724852071005921, p= 0.300384770390566, dof= 1
Ho, our null hypothesis is the two variables are independent.
p value > 0.05 so we likely cannot reject the null hypothesis 
=> independent


chi2= 19.911071844194957, p= 8.11291182934436e-06, dof= 1
Ho, our null hypothesis is the two variables are independent.
p value <= 0.05 so can lean towards rejecting the null hypothesis.
=> some dependency likely




In [37]:
# Some quick playing around first ...  using column 2 which is the "time of day" feature.
indices = np.random.choice(range(100000), replace=False, size=10000)
# X[0] # array(['Midtown East', 0, 4, 'Customer', 1], dtype=object)
table = pd.crosstab(X[indices, 2], y[indices])

#X[indices, 4]#.shape, y[indices].shape
#Counter(X[indices, 2])
# from scipy.stats import chi2_contingency

chi2, p, dof, expected = chi2_contingency(table)
print(f'chi2 {chi2}, p {p}, dof {dof}')
print(table)
print(expected)

chi2 960.8880653984245, p 5.473221121152702e-165, dof 56
col_0  Chelsea  East Village  Garment District  Gramercy Park  \
row_0                                                           
0           69           123                25             49   
1           36           128                49             44   
2           89           118                83             42   
3           84           268               109            105   
4            9            61                28             38   

col_0  Lower East Side  Midtown  Midtown East  Midtown West  Murray Hill  \
row_0                                                                      
0                   12      136           612           407          235   
1                   22      198           318           282          144   
2                   20      206           296           199          112   
3                   70      175           536           264          261   
4                   28       34

In [28]:
table

col_0,Chelsea,East Village,Garment District,Gramercy Park,Lower East Side,Midtown,Midtown East,Midtown West,Murray Hill,Nolita,Rose Hill,Stuyvesant Town,Theater District,Ukrainian Village,Union Square
row_0,,,,,,,,,,,,,,,
0,59,124,23,51,11,161,594,435,214,104,65,48,387,16,166
1,40,112,40,50,20,178,348,238,143,103,59,96,169,59,194
2,78,120,71,68,28,161,294,196,113,106,54,98,153,70,171
3,101,248,114,117,80,185,511,258,240,209,124,304,207,188,257
4,9,64,34,34,26,30,122,70,61,57,33,81,64,42,42


In [47]:
# Ok.. got for all of it... compare independent variable  at a time.. 
# indices = np.random.choice(range(100000), replace=False, size=10000)
# X[0] # array(['Midtown East', 0, 4, 'Customer', 1], dtype=object)
independent_variables = {0: 'start_neighborhood',
                        1: 'gender',
                        2: 'time_of_day',
                        3: 'usertype',
                        4: 'weekday'}
for col_index, var in independent_variables.items():
    # Build a cross-tab 
    print('====================================================')
    print(f'Building crosstab on {var}')
    table = pd.crosstab(X[:, col_index], y)
    #print(table)
    chi_sq_test(table)
    #print()



Building crosstab on start_neighborhood
chi2= 1101939.4755814166, p= 0.0, dof= 2809
Ho, our null hypothesis is the two variables are independent.
p value <= 0.05 so can lean towards rejecting the null hypothesis.
=> some dependency likely


Building crosstab on gender
chi2= 28181.371173591728, p= 0.0, dof= 106
Ho, our null hypothesis is the two variables are independent.
p value <= 0.05 so can lean towards rejecting the null hypothesis.
=> some dependency likely


Building crosstab on time_of_day
chi2= 43677.30494407689, p= 0.0, dof= 212
Ho, our null hypothesis is the two variables are independent.
p value <= 0.05 so can lean towards rejecting the null hypothesis.
=> some dependency likely


Building crosstab on usertype
chi2= 23638.087048050013, p= 0.0, dof= 53
Ho, our null hypothesis is the two variables are independent.
p value <= 0.05 so can lean towards rejecting the null hypothesis.
=> some dependency likely


Building crosstab on weekday
chi2= 9183.135943620315, p= 0.0, dof= 53


There can be much more to be said and done here, but for now I think I will casually use this as proof that I should keep going ! There's yet a model that can capture the entropy in this data!.